<a href="https://colab.research.google.com/github/bebyakinb/test_assignments_Analyst/blob/master/habr/SQL_Hardest_Interview_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Подготовка среды

In [1]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz &> /dev/null
!tar xzf sqlite.tar.gz &> /dev/null
%cd sqlite/
!./configure &> /dev/null
!make sqlite3.c &> /dev/null
%cd /content
!npx degit coleifer/pysqlite3 -f &> /dev/null
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build &> /dev/null
!cp build/lib.linux-x86_64-3.8/pysqlite3/_sqlite3.cpython-38-x86_64-linux-gnu.so \
    /usr/lib/python3.8/lib-dynload/_sqlite3.cpython-38-x86_64-linux-gnu.so

/content/sqlite
/content


In [1]:
import sqlite3
sqlite3.sqlite_version

'3.40.0'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Задача №1. Процентное изменение месяц к месяцу


## 1.0. Постановка задачи
**Контекст:** часто полезно знать, как изменяется ключевая метрика, например, месячная аудитория активных пользователей, от месяца к месяцу. Допустим у нас есть таблица logins в таком виде:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```
**Задача:** Найти ежемесячное процентное изменение месячной аудитории активных пользователей (MAU).

## 1.1. Подготовка данных

In [3]:
def random_dates(start, end, n=10):
    '''
    Dividing the unix time values(start and end) by 24*60*60*10**9
    to make it days and return n randomly chosen days back.
    '''
    start_u = start.value//(24*60*60*10**9)
    end_u = end.value//(24*60*60*10**9)

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='D')


In [4]:
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2022-12-01')
login_lines = 100000
user_ids = 30000

logins = pd.DataFrame()
logins['user_id'] = np.random.randint(1, user_ids, login_lines)
logins['date'] = pd.Series(random_dates(start, end, login_lines))
logins

,user_id,date
0,455,2022-09-18
1,25117,2022-05-24
2,9855,2021-05-23
3,20347,2022-07-12
4,24493,2021-07-07
...,...,...
99995,3257,2022-02-19
99996,27724,2022-03-19
99997,12784,2022-09-20
99998,7448,2022-07-22


In [5]:
con = sqlite3.connect('db')
cur = con.cursor()

In [6]:
logins.to_sql('logins', con, if_exists='replace',index=False)

In [7]:
def select(sql):
  return pd.read_sql(sql,con)

In [8]:
sql = '''select * from logins'''
select(sql)

,user_id,date
0,455,2022-09-18 00:00:00
1,25117,2022-05-24 00:00:00
2,9855,2021-05-23 00:00:00
3,20347,2022-07-12 00:00:00
4,24493,2021-07-07 00:00:00
...,...,...
99995,3257,2022-02-19 00:00:00
99996,27724,2022-03-19 00:00:00
99997,12784,2022-09-20 00:00:00
99998,7448,2022-07-22 00:00:00


## 1.2. Решение SQL

In [9]:
sql ='''
with months_active as (
  select strftime('%Y %m', date) as year_month,
         cast(count(user_id) as real) as users from logins
  group by year_month)

select 
year_month, users, (users - LAG(users) over(order by year_month))*100/LAG(users) over(order by year_month) MAU_percent
from months_active
'''
select(sql)

,year_month,users,MAU_percent
0,2021 01,4423.0,NaN
1,2021 02,4009.0,-9.360163
2,2021 03,4470.0,11.499127
3,2021 04,4285.0,-4.138702
4,2021 05,4370.0,1.983664
5,2021 06,4436.0,1.510297
6,2021 07,4382.0,-1.217313
7,2021 08,4482.0,2.282063
8,2021 09,4303.0,-3.993753
9,2021 10,4442.0,3.230304


## 1.3 Решение Pandas

In [10]:
pt = logins.pivot_table(index=[(logins['date'].dt.to_period('M'))], 
                        values=['user_id'],
                        aggfunc="count"
                       )
pt['MAU,%mm'] = (pt['user_id'] - pt['user_id'].shift(1))/pt['user_id'].shift(1)*100
pt

,user_id,"MAU,%mm"
date,,
2021-01,4423,NaN
2021-02,4009,-9.360163
2021-03,4470,11.499127
2021-04,4285,-4.138702
2021-05,4370,1.983664
2021-06,4436,1.510297
2021-07,4382,-1.217313
2021-08,4482,2.282063
2021-09,4303,-3.993753


# 2.Задача №2. Маркировка древовидной структуры

## 2.0 Постановка задачи
Контекст: предположим, у вас есть таблица tree с двумя столбцами: в первом указаны узлы, а во втором — родительские узлы.
```
node   parent
1       2
2       5
3       5
4       3
5       NULL 
```
Задача: написать SQL таким образом, чтобы мы обозначили каждый узел как внутренний (inner), корневой (root) или конечный узел/лист (leaf), так что для вышеперечисленных значений получится следующее:
```
node    label  
1       Leaf
2       Inner
3       Inner
4       Leaf
5       Root
```

## 2.1 Подготовка данных

In [11]:
nodes = pd.DataFrame({'node': [1,2,3,4,5],
                      'parent':[2,5,5,3,np.nan]})
nodes

,node,parent
0,1,2.0
1,2,5.0
2,3,5.0
3,4,3.0
4,5,NaN


In [12]:
nodes.to_sql('nodes', con, if_exists='replace',index=False)

### 2.2 Решение SQL

In [13]:
sql = '''
select node,
       case
            when parent is null then 'Root'
            when node in (select parent from nodes) then 'Inner'
            else 'Leaf'
       end as label
from nodes
'''
select(sql)

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


## 2.3 Решение Pandas

In [14]:
nodes['label'] =  np.where(nodes['parent'].isna(), 'Root',
                  np.where(nodes['node'].isin(nodes['parent']), 'Inner', 'Leaf'))
nodes[['node','label']]

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


# 3. Задача № 3. Удержание пользователей в месяц.

## 3.0. Постановка задачи

**Контекст:** допустим, у нас есть статистика по авторизации пользователей на сайте в таблице logins:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```

**Задача:** написать запрос, который получает количество удержанных пользователей в месяц. В нашем случае данный параметр определяется как количество пользователей, которые авторизовались в системе и в этом, и в предыдущем месяце.


## 3.1. Подготовка данных

In [15]:
logins.to_sql('logins', con, if_exists='replace',index=False)

## 3.2. Решение SQL

In [16]:
sql = '''
with months_logins as (
  select distinct user_id, 
         strftime('%Y %m', date) as cur_month,
         strftime('%Y %m', date(date, 'start of month','-1 month')) as prev_month
  from logins
  )

select m1.cur_month, count(m2.user_id)
from months_logins as m1
left join months_logins as m2 ON m1.user_id = m2.user_id AND m1.prev_month = m2.cur_month
group by m1.cur_month
'''
select(sql)

,cur_month,count(m2.user_id)
0,2021 01,0
1,2021 02,494
2,2021 03,543
3,2021 04,582
4,2021 05,549
5,2021 06,583
6,2021 07,545
7,2021 08,587
8,2021 09,534
9,2021 10,532


## 3.3. Решение Pandas

In [17]:
logins_extended = logins.copy()
logins_extended['id_and_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + logins['date'].dt.to_period('M').dt.to_timestamp().astype(str))
logins_extended['id_and_prev_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + (logins['date'].dt.to_period('M').dt.to_timestamp() 
       - pd.DateOffset(months=1)).astype(str))
logins_extended = logins_extended.drop_duplicates(['user_id','id_and_month'])
logins_extended

,user_id,date,id_and_month,id_and_prev_month
0,455,2022-09-18,455_2022-09-01,455_2022-08-01
1,25117,2022-05-24,25117_2022-05-01,25117_2022-04-01
2,9855,2021-05-23,9855_2021-05-01,9855_2021-04-01
3,20347,2022-07-12,20347_2022-07-01,20347_2022-06-01
4,24493,2021-07-07,24493_2021-07-01,24493_2021-06-01
...,...,...,...,...
99993,21369,2021-09-11,21369_2021-09-01,21369_2021-08-01
99995,3257,2022-02-19,3257_2022-02-01,3257_2022-01-01
99996,27724,2022-03-19,27724_2022-03-01,27724_2022-02-01
99997,12784,2022-09-20,12784_2022-09-01,12784_2022-08-01


In [18]:
logins_extended['prev_month_login'] = (
    np.where(logins_extended['id_and_prev_month'].isin(logins_extended['id_and_month'])
             , 1 , 0))
logins_extended.groupby([logins_extended['date'].dt.to_period('M')])['prev_month_login'].sum()

<ipython-input-18-51ef2d549242>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logins_extended['prev_month_login'] = (


date
2021-01      0
2021-02    494
2021-03    543
2021-04    582
2021-05    549
2021-06    583
2021-07    545
2021-08    587
2021-09    534
2021-10    532
2021-11    566
2021-12    553
2022-01    541
2022-02    495
2022-03    538
2022-04    541
2022-05    545
2022-06    546
2022-07    554
2022-08    540
2022-09    499
2022-10    540
2022-11    575
Freq: M, Name: prev_month_login, dtype: int64

# 4. Задача № 4. Нарастающий итог

## 4.0. Поставнока задачи

**Контекст:** допустим, у нас есть таблица transactions в таком виде:

| date       | cash_flow |
|------------|-----------|
| 2018-01-01 | -1000     |
| 2018-01-02 | -100      |
| 2018-01-03 | 50        |
| ...        | ...       |


Где cash_flow — это выручка минус затраты за каждый день.

**Задача:** написать запрос, чтобы получить нарастающий итог для денежного потока каждый день таким образом, чтобы в конечном итоге получилась таблица в такой форме:

| date       | cumulative_cf |
|------------|---------------|
| 2018-01-01 | -1000         |
| 2018-01-02 | -1100         |
| 2018-01-03 | -1050         |
| ...        | ...           |

## 4.1. Подготовка данных

In [26]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

transactions = pd.DataFrame()
transactions['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
transactions['cash_flow'] = np.random.randint(-2000, 2000, sample_size)
transactions

,date,cash_flow
0,2021-01-01,495
1,2021-01-02,1649
2,2021-01-03,-172
3,2021-01-04,1105
4,2021-01-05,1181
5,2021-01-06,-1320
6,2021-01-07,1717
7,2021-01-08,406
8,2021-01-09,1394
9,2021-01-10,1511


In [27]:
transactions.to_sql('transactions', con, if_exists='replace',index=False)

## 4.2. Решение SQL

In [28]:
sql = '''
select
  date,
  sum(cash_flow) over (order by date) AS cumulative_cf
from transactions
group by date
order by date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,495
1,2021-01-02 00:00:00,2144
2,2021-01-03 00:00:00,1972
3,2021-01-04 00:00:00,3077
4,2021-01-05 00:00:00,4258
5,2021-01-06 00:00:00,2938
6,2021-01-07 00:00:00,4655
7,2021-01-08 00:00:00,5061
8,2021-01-09 00:00:00,6455
9,2021-01-10 00:00:00,7966


## 4.3. Решение SQL без оконной функции

In [29]:
sql = '''
select
  t1.date,
  sum(t2.cash_flow) as cumulative_cf
from transactions as t1
join transactions as t2 on t1.date >= t2.date
group by t1.date
order by t1.date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,495
1,2021-01-02 00:00:00,2144
2,2021-01-03 00:00:00,1972
3,2021-01-04 00:00:00,3077
4,2021-01-05 00:00:00,4258
5,2021-01-06 00:00:00,2938
6,2021-01-07 00:00:00,4655
7,2021-01-08 00:00:00,5061
8,2021-01-09 00:00:00,6455
9,2021-01-10 00:00:00,7966


## 4.4. Решение Pandas

In [30]:
transactions_extended = transactions.copy().sort_values('date')
transactions_extended['cumulative_cf'] = transactions_extended['cash_flow'].cumsum()
transactions_extended[['date','cumulative_cf']]

,date,cumulative_cf
0,2021-01-01,495
1,2021-01-02,2144
2,2021-01-03,1972
3,2021-01-04,3077
4,2021-01-05,4258
5,2021-01-06,2938
6,2021-01-07,4655
7,2021-01-08,5061
8,2021-01-09,6455
9,2021-01-10,7966


# 5. Задача № 5. Скользящее среднее

## 5.0. Постановка задачи

**Контекст:** допустим, у нас есть таблица signups в таком виде:

| date       | sign_ups |
|------------|----------|
| 2018-01-01 | 10       |
| 2018-01-02 | 20       |
| 2018-01-03 | 50       |
| ...        | ...      |
| 2018-10-01 | 35       |


**Задача:** написать запрос, чтобы получить 7-дневное скользящее среднее ежедневных регистраций

## 5.1. Подготовка данных

In [31]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

signups = pd.DataFrame()
signups['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
signups['sign_ups'] = np.random.randint(0, 100, sample_size)
signups

,date,sign_ups
0,2021-01-01,85
1,2021-01-02,49
2,2021-01-03,33
3,2021-01-04,60
4,2021-01-05,77
5,2021-01-06,5
6,2021-01-07,4
7,2021-01-08,0
8,2021-01-09,35
9,2021-01-10,3


In [32]:
signups.to_sql('signups', con, if_exists='replace',index=False)

## 5.2. Решение SQL

In [33]:
sql = '''
select
  s1.date,
  s1.sign_ups,
  avg(s2.sign_ups)
from signups as s1

join signups as s2 on s1.date < date(s2.date, '7 days') and s1.date >= s2.date
group by s1.date
'''

results = select(sql)
results

,date,sign_ups,avg(s2.sign_ups)
0,2021-01-01 00:00:00,85,85.000000
1,2021-01-02 00:00:00,49,67.000000
2,2021-01-03 00:00:00,33,55.666667
3,2021-01-04 00:00:00,60,56.750000
4,2021-01-05 00:00:00,77,60.800000
5,2021-01-06 00:00:00,5,51.500000
6,2021-01-07 00:00:00,4,44.714286
7,2021-01-08 00:00:00,0,32.571429
8,2021-01-09 00:00:00,35,30.571429
9,2021-01-10 00:00:00,3,26.285714


## 5.3. Решение Pandas

In [34]:
signups_extended = signups.copy().set_index('date')
signups_extended.rolling('7D').mean()

,sign_ups
date,
2021-01-01,85.000000
2021-01-02,67.000000
2021-01-03,55.666667
2021-01-04,56.750000
2021-01-05,60.800000
2021-01-06,51.500000
2021-01-07,44.714286
2021-01-08,32.571429
2021-01-09,30.571429


# 6. Задача № 6. Несколько условий соединения

## 6.0 Постановка задачи

**Контекст:** скажем, наша таблица emails содержит электронные письма, отправленные с адреса zach@g.com и полученные на него:

| id | subject  | from         | to           | timestamp           |
|----|----------|--------------|--------------|---------------------|
| 1  | Yosemite | zach@g.com   | thomas@g.com | 2018-01-02 12:45:03 |
| 2  | Big Sur  | sarah@g.com  | thomas@g.com | 2018-01-02 16:30:01 |
| 3  | Yosemite | thomas@g.com | zach@g.com   | 2018-01-02 16:35:04 |
| 4  | Running  | jill@g.com   | zach@g.com   | 2018-01-03 08:12:45 |
| 5  | Yosemite | zach@g.com   | thomas@g.com | 2018-01-03 14:02:01 |
| 6  | Yosemite | thomas@g.com | zach@g.com   | 2018-01-03 15:01:05 |
| .. | ..       | ..           | ..           | ..                  |


**Задача:** написать запрос, чтобы получить время отклика на каждое письмо (id), отправленное на zach@g.com. Не включать письма на другие адреса. Предположим, что у каждого треда уникальная тема. Имейте в виду, что в треде может быть несколько писем туда и обратно между zach@g.com и другими адресатами.


## 6.1 Подготовка данных

In [35]:
emails = pd.DataFrame({'subject': ['Yosemite',
                                   'Big Sur',
                                   'Yosemite',
                                   'Running',
                                   'Yosemite',
                                   'Yosemite'],
                       'from' : ['zach@g.com',
                                 'sarah@g.com',
                                 'thomas@g.com',
                                 'jill@g.com',
                                 'zach@g.com',
                                 'thomas@g.com'],
                       'to' : ['thomas@g.com',
                               'thomas@g.com',
                               'zach@g.com',
                               'zach@g.com',
                               'thomas@g.com',
                               'zach@g.com'],
                       'date': [pd.Timestamp('2018-01-02 12:45:03'),
                                pd.Timestamp('2018-01-02 16:30:01'),
                                pd.Timestamp('2018-01-02 16:35:04'),
                                pd.Timestamp('2018-01-03 08:12:45'),
                                pd.Timestamp('2018-01-03 14:02:01'),
                                pd.Timestamp('2018-01-03 15:01:05')]
                       })
emails

,subject,from,to,date
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03
1,Big Sur,sarah@g.com,thomas@g.com,2018-01-02 16:30:01
2,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04
3,Running,jill@g.com,zach@g.com,2018-01-03 08:12:45
4,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01
5,Yosemite,thomas@g.com,zach@g.com,2018-01-03 15:01:05


In [36]:
emails.to_sql('emails', con, if_exists='replace')

## 6.2. Решение SQL

In [37]:
sql = '''
select e1.subject,
       e1."from",
       e1."to",
       e1.date,
       e2.date,
       CAST((JulianDay(min(e2.date)) - JulianDay(e1.date)) * 24 as Integer) as diff_in_hours
from emails as e1
join emails as e2 on e1.subject = e2.subject and 
                     e1."from" = e2."to" and
                     e1."to" = e2."from" and
                     e1.date < e2.date
--where e1."to" == "zach@g.com"
group by e1."index"
'''

select(sql)

,subject,from,to,date,date,diff_in_hours
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03,2018-01-02 16:35:04,3
1,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04,2018-01-03 14:02:01,21
2,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01,2018-01-03 15:01:05,0


## 6.3. Решение Pandas

In [38]:
emails_extended = pd.merge(emails,
                           emails,
                           how='left',
                           left_on=['subject','from','to'],
                           right_on = ['subject','to','from']
                          ).query('date_x < date_y')
emails_extended['diff_in_hours'] = (emails_extended['date_y'] -
                                    emails_extended['date_x']
                                   )
emails_extended.groupby(['subject',
                         'from_x',
                         'to_x',
                         'date_x'
                        ]).agg({'date_y':'min',
                                'diff_in_hours':'min'
                               }).sort_values(['date_x']).reset_index()

,subject,from_x,to_x,date_x,date_y,diff_in_hours
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03,2018-01-02 16:35:04,0 days 03:50:01
1,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04,2018-01-03 14:02:01,0 days 21:26:57
2,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01,2018-01-03 15:01:05,0 days 00:59:04


# 7. Задача № 7. Найти идентификатор с максимальным значением

## 7.0. Постановка задачи

**Контекст:** Допустим, у нас есть таблица salaries с данными об отделах и зарплате сотрудников в следующем формате:

|  depname  | empno | salary |     
|-----------|-------|--------|
| develop   |    11 |   5200 | 
| develop   |     7 |   4200 | 
| develop   |     9 |   4500 | 
| develop   |     8 |   6000 | 
| develop   |    10 |   5200 | 
| personnel |     5 |   3500 | 
| personnel |     2 |   3900 | 
| sales     |     3 |   4800 | 
| sales     |     1 |   5000 | 
| sales     |     4 |   4800 |

**Задача:** написать запрос, чтобы получить empno с самой высокой зарплатой. Убедитесь, что ваше решение обрабатывает случаи одинаковых зарплатами!


## 7.1. Подготовка данных

In [63]:
sample_size = 10
salaries = pd.DataFrame()
salaries['depname'] = np.random.choice(['develop','personnel','sales'],
                                       sample_size)
salaries['empno'] = np.random.choice(range(sample_size),
                                       sample_size,
                                       replace=False)
salaries['salary'] = np.random.randint(4000, 12000, sample_size)
salaries

,depname,empno,salary
0,sales,6,5704
1,sales,2,10009
2,sales,5,10332
3,develop,4,10670
4,personnel,3,9636
5,sales,7,5363
6,develop,8,7765
7,personnel,1,5069
8,sales,0,10939
9,personnel,9,6171


In [64]:
salaries.to_sql('salaries', con, if_exists='replace', index=False)

## 7.2. Решение SQL

In [60]:
sql = '''
with max_salary as (
  select max(salary) as max_salary
  from salaries
)

select empno from salaries
join max_salary on salaries.salary = max_salary.max_salary
'''
select(sql)

,empno
0,7
1,8
2,0


## 7.3. Решение Pandas

In [61]:
salaries[salaries['salary'] == salaries['salary'].max()]['empno']

0    7
7    8
9    0
Name: empno, dtype: int64

# 8. Задача № 8. Среднее значение и ранжирование с оконной функцией

## 8.0. Постановка задачи

**Задача:** спользую наббор данных из предыдушей задачи написать запрос, который возвращает ту же таблицу, но с новым столбцом, в котором указана средняя зарплата по департаменту. Мы бы ожидали таблицу в таком виде:

  depname  | empno | salary | avg_salary |     
-----------|-------|--------|------------|
 develop   |    11 |   5200 |       5020 |
 develop   |     7 |   4200 |       5020 | 
 develop   |     9 |   4500 |       5020 |
 develop   |     8 |   6000 |       5020 | 
 develop   |    10 |   5200 |       5020 | 
 personnel |     5 |   3500 |       3700 |
 personnel |     2 |   3900 |       3700 |
 sales     |     3 |   4800 |       4867 | 
 sales     |     1 |   5000 |       4867 | 
 sales     |     4 |   4800 |       4867 |

## 8.1. Решение SQL

In [70]:
sql = '''
select *, avg(salary) OVER (PARTITION BY depname) as avg_salary
from salaries
'''
select(sql)

,depname,empno,salary,avg_salary
0,develop,4,10670,9217.500000
1,develop,8,7765,9217.500000
2,personnel,3,9636,6958.666667
3,personnel,1,5069,6958.666667
4,personnel,9,6171,6958.666667
5,sales,6,5704,8469.400000
6,sales,2,10009,8469.400000
7,sales,5,10332,8469.400000
8,sales,7,5363,8469.400000
9,sales,0,10939,8469.400000


## 8.2. Решение Pandas

In [79]:
salaries_extended = salaries.copy()
salaries_extended['avg_price'] = salaries_extended.groupby(['depname'])['salary'].transform('mean')
salaries_extended.sort_values('depname')

,depname,empno,salary,avg_price
3,develop,4,10670,9217.500000
6,develop,8,7765,9217.500000
4,personnel,3,9636,6958.666667
7,personnel,1,5069,6958.666667
9,personnel,9,6171,6958.666667
0,sales,6,5704,8469.400000
1,sales,2,10009,8469.400000
2,sales,5,10332,8469.400000
5,sales,7,5363,8469.400000
8,sales,0,10939,8469.400000
